- Criar um dataframe contendo informações de risco de roubo
- Avaliar o risco da estação(baixo, medio, alto)
    score < 500 = low
    score > 500 & < 700 = medium
    score > 700 = high

In [13]:
file_path="/home/jovyan/work/data/202212_divvy_tripdata.csv"
score_path = "/home/jovyan/work/data/station_scores.csv"


from pyspark.sql import SparkSession
from pyspark.sql.functions import when

spark = SparkSession.builder \
    .appName("aula-05") \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

df_trip = spark.read.option("header", "true").csv(file_path)

df_bike_share = df_trip.withColumn('end_station_name', when(df_trip['end_station_name'].isNull(), 'unknown').otherwise(df_trip['end_station_name']))

In [14]:
df_station_score = spark.read.option("header", "true").csv(station_score_path)
display(df_station_score)

end_station_id,score
KA1504000162,404
15634,653
691,394
13192,234
15539,237
KA1503000075,313
TA1305000022,799
20244,667
KA1504000135,444
TA1305000009,470


In [16]:
df_station_scores = df_bike_share.join(df_station_score, on="end_station_id", how="inner")
display(df_station_scores)

end_station_id,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,score
13191,65DBD2F447EC51C2,electric_bike,2022-12-05 10:47:18,2022-12-05 10:56:34,Clifton Ave & Arm...,TA1307000163,Sedgwick St & Web...,41.9182435,-87.65711466666667,41.922167,-87.638888,member,29
13191,0C201AA7EA0EA1AD,classic_bike,2022-12-18 06:42:33,2022-12-18 07:08:44,Broadway & Belmon...,13277,Sedgwick St & Web...,41.940106,-87.645451,41.922167,-87.638888,casual,29
13016,E0B148CCB358A49D,electric_bike,2022-12-13 08:47:45,2022-12-13 08:59:51,Sangamon St & Lak...,TA1306000015,St. Clair St & Er...,41.885918856,-87.65113318,41.89434513742426,-87.62279838323593,member,884
13134,54C5775D2B7C9188,classic_bike,2022-12-13 18:50:47,2022-12-13 19:19:48,Shields Ave & 31s...,KA1503000038,Damen Ave & Madis...,41.838464,-87.635406,41.88137,-87.67493,member,794
13288,A4891F78776D35DF,classic_bike,2022-12-14 16:13:39,2022-12-14 16:27:50,Ashland Ave & Chi...,13247,Damen Ave & Charl...,41.89595435734305,-87.6677280664444,41.920082,-87.677855,casual,92
KA1503000072,DB91D9B8DFACA07A,electric_bike,2022-12-02 15:24:47,2022-12-02 15:34:14,Wabash Ave & 9th St,TA1309000010,Wacker Dr & Washi...,41.870683551,-87.625706077,41.88314336439022,-87.63724207878113,member,503
KA1503000072,6AD396C5760CC992,classic_bike,2022-12-20 09:02:03,2022-12-20 09:19:54,Leavitt St & Chic...,18058,Wacker Dr & Washi...,41.895501,-87.682017,41.88314336439022,-87.63724207878113,member,503
15650,8D736E35E0075504,classic_bike,2022-12-13 14:19:49,2022-12-13 14:28:28,Humboldt Blvd & A...,15651,Spaulding Ave & A...,41.91751339399,-87.7018089555,41.917108034789315,-87.71022096276283,member,609
13016,DDAB881F96C51DC8,classic_bike,2022-12-13 07:38:07,2022-12-13 07:57:01,Canal St & Adams St,13011,St. Clair St & Er...,41.879255,-87.639904,41.89434513742426,-87.62279838323593,member,884
13235,8DDBAE51E55DAEEF,classic_bike,2022-12-21 08:57:42,2022-12-21 09:10:59,Greenview Ave & F...,TA1307000001,Southport Ave & W...,41.92533,-87.6658,41.94815,-87.66394,member,922


In [17]:
df_station_scores.withColumn("risk",
                            when((df_station_scores["score"] < 500), "low")
                            .when((df_station_scores["score"] > 500) & (df_station_scores["score"] < 700), "medium")
                            .when((df_station_scores["score"] > 700), "high")
                            )

end_station_id,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,score,risk
13191,65DBD2F447EC51C2,electric_bike,2022-12-05 10:47:18,2022-12-05 10:56:34,Clifton Ave & Arm...,TA1307000163,Sedgwick St & Web...,41.9182435,-87.65711466666667,41.922167,-87.638888,member,29,low
13191,0C201AA7EA0EA1AD,classic_bike,2022-12-18 06:42:33,2022-12-18 07:08:44,Broadway & Belmon...,13277,Sedgwick St & Web...,41.940106,-87.645451,41.922167,-87.638888,casual,29,low
13016,E0B148CCB358A49D,electric_bike,2022-12-13 08:47:45,2022-12-13 08:59:51,Sangamon St & Lak...,TA1306000015,St. Clair St & Er...,41.885918856,-87.65113318,41.89434513742426,-87.62279838323593,member,884,high
13134,54C5775D2B7C9188,classic_bike,2022-12-13 18:50:47,2022-12-13 19:19:48,Shields Ave & 31s...,KA1503000038,Damen Ave & Madis...,41.838464,-87.635406,41.88137,-87.67493,member,794,high
13288,A4891F78776D35DF,classic_bike,2022-12-14 16:13:39,2022-12-14 16:27:50,Ashland Ave & Chi...,13247,Damen Ave & Charl...,41.89595435734305,-87.6677280664444,41.920082,-87.677855,casual,92,low
KA1503000072,DB91D9B8DFACA07A,electric_bike,2022-12-02 15:24:47,2022-12-02 15:34:14,Wabash Ave & 9th St,TA1309000010,Wacker Dr & Washi...,41.870683551,-87.625706077,41.88314336439022,-87.63724207878113,member,503,medium
KA1503000072,6AD396C5760CC992,classic_bike,2022-12-20 09:02:03,2022-12-20 09:19:54,Leavitt St & Chic...,18058,Wacker Dr & Washi...,41.895501,-87.682017,41.88314336439022,-87.63724207878113,member,503,medium
15650,8D736E35E0075504,classic_bike,2022-12-13 14:19:49,2022-12-13 14:28:28,Humboldt Blvd & A...,15651,Spaulding Ave & A...,41.91751339399,-87.7018089555,41.917108034789315,-87.71022096276283,member,609,medium
13016,DDAB881F96C51DC8,classic_bike,2022-12-13 07:38:07,2022-12-13 07:57:01,Canal St & Adams St,13011,St. Clair St & Er...,41.879255,-87.639904,41.89434513742426,-87.62279838323593,member,884,high
13235,8DDBAE51E55DAEEF,classic_bike,2022-12-21 08:57:42,2022-12-21 09:10:59,Greenview Ave & F...,TA1307000001,Southport Ave & W...,41.92533,-87.6658,41.94815,-87.66394,member,922,high
